**Image unimodal for hateful memes**

Steps:
1. Unzip hateful_memes.zip and upload unzipped hateful_memes folder to your google drive and change your data_dir to the path of your uploaded hateful_memes folder
2. Run the "Installation" section
3. Run "Build & Train Model" section 

Note: For step 2, there are two sets of test groups, as shown in the last block of "Build & Train Model" section. The needed values of vision_feature_dim in hparam are different in the two test groups. I commented right after each model for the needed value of vision_feature_dim, i.e. (a) for test1 group's models, we need to set it to 1000; e.g. "vision_feature_dim": 1000, (b) while for test2 group's mdoes, we need to set it to 300, i.e.      "vision_feature_dim": 300


### Installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas_path pytorch-lightning transformers spacy ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 38.0 MB/s 
     |████████████████████████████████| 5.8 MB 59.7 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 512 kB 72.2 MB/s 
     |████████████████████████████████| 125 kB 69.9 MB/s 
     |████████████████████████████████| 7.6 MB 62.3 MB/s 
     |████████████████████████████████| 182 kB 81.6 MB/s 


In [ ]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 23.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=6d2116bd1f647cb3a82e9f9cb5b604ec0d9d2638a62128d8bf11a2faf4eb2ca4
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
!pip install pandas-path

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip freeze

absl-py==1.3.0
aeppl==0.0.33
aesara==2.7.9
aiohttp==3.8.3
aiosignal==1.3.1
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
appdirs==1.4.4
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
atari-py==0.2.9
atomicwrites==1.4.1
attrs==22.1.0
audioread==3.0.0
autograd==1.5
Babel==2.11.0
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.9
bokeh==2.3.3
branca==0.6.0
bs4==0.0.1
CacheControl==0.12.11
cachetools==5.2.0
catalogue==2.0.8
certifi==2022.9.24
cffi==1.15.1
cftime==1.6.2
chardet==3.0.4
charset-normalizer==2.1.1
click==7.1.2
clikit==0.6.2
cloudpickle==1.5.0
cmake==3.22.6
cmdstanpy==1.0.8
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.3
cons==0.4.5
contextlib2==0.5.5
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cufflinks==0.17.3
cupy-cuda11x==11.0.0
cvxopt==1.3.0
cvxpy==1.2.2
cycler==0.11.0
cymem==2.0.7
Cython==0.29.32
daft==0.0.4
dask==2022.2.1
datascience==0.17.5
db-dtypes==1.0.4
debugpy==1.0.0
decorator==4.4.2
d

In [ ]:
%matplotlib inline

from pathlib import Path
from tqdm import tqdm

import random
import tarfile
import tempfile
import warnings
import matplotlib.pyplot as plt
import numpy as np
import json
import logging
import pandas as pd
import pandas_path
import pytorch_lightning as pl
import torch
import torch.nn as nn                      
import torchvision
from pandas_path import path
from PIL import Image
from transformers import AutoModel, AutoTokenizer
from glob import glob
import os
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

**check gpu**

In [ ]:
torch.cuda.is_available()

True

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e74d2109-7656-91bd-da43-33e9303bcc7f)


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec 15 02:20:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    29W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**check memory**

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:

# for the purposes of this post, we'll filter
# much of the lovely logging info from our LightningModule
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.WARNING)

In [ ]:
ls

drive/  sample_data/


In [ ]:
data_dir = Path("/content/drive/MyDrive/CS7643_final_project/hateful_memes")
print(data_dir)

train_path = data_dir / "train.jsonl"
dev_path = data_dir / "dev_seen.jsonl"
dev_unseen_path = data_dir / "dev_unseen.jsonl"
test_path = data_dir / "test_seen.jsonl"

/content/drive/MyDrive/CS7643_final_project/hateful_memes


### **Build & Train Model**

In [ ]:
class HMDataset(torch.utils.data.Dataset):
    # Data set preprocess

    def __init__(
        self,
        data_path,
        image_path,
        image_transform,
        enforce_balance=False,
        random_seed=0,
    ):

        self.df = pd.read_json(
            data_path, lines=True
        )
        if enforce_balance:
            pos = self.df[
                self.df.label.eq(1)
            ]
            neg = self.df[
                self.df.label.eq(0)
            ]
            self.df = pd.concat(
                [
                    neg.sample(
                        pos.shape[0], 
                        random_seed=random_seed
                    ), 
                    pos
                ]
            )
        self.df = self.df.reset_index(
            drop=True
        )
        self.df.img = self.df.apply(
            lambda row: (image_path / row.img), axis=1
        )

        if not self.df.img.path.exists().all():
            raise FileNotFoundError
        if not self.df.img.path.is_file().all():
            raise TypeError
            
        self.image_transform = image_transform

In [ ]:
class ModalConcat(nn.Module):
    def __init__(
        self,
        num_classes,
        loss_fn,
        image_model,
        image_embedding_dimension,
        fusion_size,
    ):
        super(ModalConcat, self).__init__()
        self.image_model = image_model
        input_size = image_embedding_dimension
        self.fusion_layer = nn.Linear(
            in_features=input_size, 
            out_features=fusion_size
        )
        self.fc_layer = nn.Linear(
            in_features=fusion_size, 
            out_features=num_classes
        )
        self.loss_fn = loss_fn
        
    def forward(self, image, label=None):
        image_features = nn.functional.relu(
            self.image_model(image)
        )
        fuse_output = nn.functional.relu(
            self.fusion_layer(image_features)
            )
        logits = self.fc_layer(fuse_output)
        prediction = nn.functional.softmax(logits)
        loss = (
            self.loss_fn(prediction, label) 
            if label is not None else label
        )
        return (prediction, loss)

In [ ]:
class HMModal(pl.LightningModule):
    def __init__(self, hyperparameters):
        for data_key in ["train_path", "dev_path", "image_path"]:
            if data_key not in hyperparameters.keys():
                raise KeyError(
                    f"{data_key} is required in hyperparameters"
                )
        
        super(HMModal, self).__init__()
        self.hyperparameters = hyperparameters

        self.image_model_type = self.hyperparameters.get(
            "image_model_type", "resnet152"
        )
        self.image_model = self.fetch_image_model(self.image_model_type)
        self.image_embedding_dimension = self.hyperparameters.get(
            "image_embedding_dimension", 128
        )
        self.output_path = Path(
            self.hyperparameters.get("output_path", "model_output")
        )
        self.output_path.mkdir(exist_ok=True)
    
        self.image_transform = self.transform_image()
        self.training_dataset = self.create_dataset("train_path")
        self.dev_dataset = self.create_dataset("dev_path")
        
        self.model = self.create_model()
        self.trainer_parameters = self.fetch_trainer_parameters()
    
    def forward(self, image, label=None):
        return self.model(image, label)

    def seed(self, seed):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

    def transform_image(self):
        image_dim = self.hyperparameters.get("image_dim", 224)
        image_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize(
                    size=(image_dim, image_dim)
                ),        
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(
                    mean=(0.485, 0.456, 0.406), 
                    std=(0.229, 0.224, 0.225)
                ),
            ]
        )
        return image_transform

    def create_dataset(self, dataset_key):
        return HMDataset(
            data_path=self.hyperparameters.get(dataset_key, dataset_key),
            image_path=self.hyperparameters.get("image_path"),
            image_transform=self.image_transform,
            enforce_balance=True if "train" in str(dataset_key) else False,
        )

    def fetch_image_model(self, model_name):
        image_model_map = {
            "alexnet" : torchvision.models.alexnet,
            "densenet161" : torchvision.models.densenet161,
            "resnet152" : torchvision.models.resnet152,
            "resnext101_32x8d" : torchvision.models.resnext101_32x8d,
            "shufflenet_v2_x2_0" : torchvision.models.shufflenet_v2_x2_0,
            "vgg19_bn" : torchvision.models.vgg19_bn,
            "wide_resnet50_2" : torchvision.models.wide_resnet50_2,
            "wide_resnet101_2" : torchvision.models.wide_resnet101_2,
        }
        return image_model_map.get(model_name)
    
    def create_model(self):
        image_model = self.image_model(
            pretrained=True
        )
        image_model.fc = nn.Linear(
                in_features=2048,
                out_features=self.image_embedding_dimension
        )

        return ModalConcat(
            num_classes=self.hyperparameters.get("num_classes", 2),
            loss_fn=nn.CrossEntropyLoss(),
            image_model=image_model,
            image_embedding_dimension=self.image_embedding_dimension,
            fusion_size=self.hyperparameters.get(
                "fusion_size", 512
            ),
        )

    def train_dataloader(self):        
        return torch.utils.data.DataLoader(
            self.training_dataset, 
            shuffle=True, 
            batch_size=self.hyperparameters.get("batch_size", 5), 
            num_workers=self.hyperparameters.get("num_workers", 20)
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.dev_dataset, 
            shuffle=False, 
            batch_size=self.hyperparameters.get("batch_size", 5), 
            num_workers=self.hyperparameters.get("num_workers", 20)
        )

    def training_step(self, batch, batch_nb):
        predictions, loss = self.forward(
            image=batch["image"], 
            label=batch["label"]
        )
        
        return {"loss": loss}

    def validation_step(self, batch, batch_nb):
        predictions, loss = self.eval().forward(
            image=batch["image"], 
            label=batch["label"]
        )
        
        return {"batch_val_loss": loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack(
            tuple(
                output["batch_val_loss"] 
                for output in outputs
            )
        ).mean()

        self.log("avg_val_loss", avg_loss)
        
        return {
            "val_loss": avg_loss,
            "progress_bar":{"avg_val_loss": avg_loss}
        }

    def configure_optimizers(self):
        optimizers = [
            torch.optim.AdamW(
                self.model.parameters(), 
                lr=self.hyperparameters.get("lr", 0.001)
            )
        ]

        scheduler = {
            'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizers[0]
            ),
            'reduce_on_plateau': True,
            'monitor': 'avg_val_loss'
        }

        schedulers = [ scheduler ]
        return optimizers, schedulers
    
    def fit(self):
        self.seed(self.hyperparameters.get("random_seed", 101))
        self.trainer = pl.Trainer(**self.trainer_parameters)
        self.trainer.fit(self)
    
    def fetch_trainer_parameters(self):
        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            dirpath=self.output_path,
            monitor=self.hyperparameters.get(
                "checkpoint_monitor", "avg_val_loss"
            ),
            mode=self.hyperparameters.get(
                "checkpoint_monitor_mode", "min"
            ),
            verbose=self.hyperparameters.get("verbose", True)
        )

        early_stop_callback = pl.callbacks.EarlyStopping(
            monitor=self.hyperparameters.get(
                "early_stop_monitor", "avg_val_loss"
            ),
            min_delta=self.hyperparameters.get(
                "early_stop_min_delta", 0.0001
            ),
            patience=self.hyperparameters.get(
                "early_stop_patience", 1
            ),
            verbose=self.hyperparameters.get("verbose", True),
        )

        trainer_parameters = {
            "enable_checkpointing": True,
            "callbacks": [checkpoint_callback, early_stop_callback],
            "default_root_dir": self.output_path,
            "accumulate_grad_batches": self.hyperparameters.get(
                "accumulate_grad_batches", 1
            ),
            "auto_select_gpus": self.hyperparameters.get("auto_select_gpus", False),
            "devices": self.hyperparameters.get("devices", 1),
            "accelerator": self.hyperparameters.get("accelerator", "cpu"),
            "max_epochs": self.hyperparameters.get("max_epochs", 1000),
            "gradient_clip_val": self.hyperparameters.get(
                "gradient_clip_value", 1
            )
        }
        return trainer_parameters

    @torch.no_grad()
    def eval_accuracy(self, test_path):
        testing_dataset = self.create_dataset(test_path)
        data_frame = pd.DataFrame(
            index=testing_dataset.df.id,
            columns=["proba", "label", "target"]
        )
        test_dataloader = torch.utils.data.DataLoader(
            testing_dataset, 
            shuffle=False, 
            batch_size=self.hyperparameters.get("batch_size", 5), 
            num_workers=self.hyperparameters.get("num_workers", 20))
        for batch in tqdm(test_dataloader, total=len(test_dataloader)):
            predictions, _ = self.model.eval().to("cpu")(
                batch["image"]
            )
            data_frame.loc[batch["id"], "proba"] = predictions[:, 1]
            data_frame.loc[batch["id"], "label"] = predictions.argmax(dim=1)
            data_frame.loc[batch["id"], "target"] = batch["label"]
        data_frame.proba = data_frame.proba.astype(float)        
        data_frame.label = data_frame.label.astype(int)
        data_frame.target = data_frame.target.astype(int)
        return data_frame

In [ ]:
# test 1 with image_embedding_dimension: 1000 
test_set = [
      "alexnet",  # image_embedding_dimension: 1000 
      "densenet161",  # image_embedding_dimension: 1000 
      "vgg19_bn",  # image_embedding_dimension: 1000 
      "resnext101_32x8d",  # image_embedding_dimension: 1000 
      "wide_resnet50_2",  # image_embedding_dimension: 1000 
      "wide_resnet101_2",  # image_embedding_dimension: 1000 
      "resnet152",  # image_embedding_dimension: 300
      "resnext101_32x8d",  # image_embedding_dimension: 300
      "shufflenet_v2_x2_0",  # image_embedding_dimension: 300 
      "wide_resnet50_2",  # image_embedding_dimension: 300 
      "wide_resnet101_2",  # image_embedding_dimension: 300 
    ]

for image_model_type in test_set:
    _, accuracy, auc_roc, f1 = gen_accuracy(image_model_type)
    print("\n model: {}, \n accuracy: {}, \n auc_roc: {}, \n f1 score: {}".format(image_model_type, accuracy, auc_roc, f1))

In [ ]:
# test set 1 with image_embedding_dimension: 1000 
test_set = [
      "alexnet",  # image_embedding_dimension: 1000 
      "densenet161",  # image_embedding_dimension: 1000 
      "vgg19_bn",  # image_embedding_dimension: 1000 
      "resnext101_32x8d",  # image_embedding_dimension: 1000 
      "wide_resnet50_2",  # image_embedding_dimension: 1000 
      "wide_resnet101_2",  # image_embedding_dimension: 1000 
      "resnet152",  # image_embedding_dimension: 300
      "resnext101_32x8d",  # image_embedding_dimension: 300
      "shufflenet_v2_x2_0",  # image_embedding_dimension: 300 
      "wide_resnet50_2",  # image_embedding_dimension: 300 
      "wide_resnet101_2",  # image_embedding_dimension: 300 
    ]

for image_model_type in test_set:
    _, accuracy, auc_roc, f1 = gen_accuracy(image_model_type)
    print("\n model: {}, \n accuracy: {}, \n auc_roc: {}, \n f1 score: {}".format(image_model_type, accuracy, auc_roc, f1))